In [11]:
import minsearch
import json

In [12]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [13]:
docs_raw[1]['course']

'machine-learning-zoomcamp'

In [14]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)


In [15]:
documents[0].keys()

dict_keys(['text', 'section', 'question', 'course'])

In [16]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [34]:
index.fit(documents)

In [37]:
from openai import OpenAI

client = OpenAI()

In [47]:
def search(query):
    boost = {"question": 3.0, "section": 0.5}

    results = index.search(
        query=query,
        filter_dict={"course": "data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=5
    )

    return results

In [54]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
Use only the facts from the CONTEXT when answering the QUESTION. 
If the CONTEXT doesn't have answer, output NONE.

QUESTION: {question}

CONTEXT: {context}

    """

    context = "\n"

    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [57]:
def llm(prompt):
    response = client.chat.completions.create(
                    model= "gpt-3.5-turbo-0125",
                    messages=[{"role": "user",
                            "content": prompt}]
                )
    
    return response.choices[0].message.content

In [60]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [61]:
q = "how do i run kafka?"

answer = rag(q)

print(answer)

To run Kafka, you can use the command: java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java as mentioned in Module 6 under streaming with kafka.
